In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import Window
import yaml, os

from utils.spark_path import get_latest_year_month_path, get_current_year_month_path

In [8]:
# ============================================================
# Spark
# ============================================================
spark = (
    SparkSession.builder
    .appName("silver_clean_toji")
    .master("spark://spark-master:7077")
    .config("spark.sql.adaptive.enabled", "true")
    .config("spark.sql.shuffle.partitions", "200")
    .getOrCreate()
)

# Config

In [9]:
CONFIG_PATH = "./config.yaml"

with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    cfg = yaml.safe_load(f)

ROOT = cfg["data_lake"]["root"]
LAYERS = cfg["data_lake"]["layers"]

# input (bronze)
toji_src_path = get_latest_year_month_path(spark, os.path.join(
    ROOT,
    LAYERS["bronze"]["domains"]["tojiSoyuJeongbo"]["paths"]["parquet"]
))                                  

# output (silver clean)
toji_clean_path = get_current_year_month_path(os.path.join(
    ROOT,
    LAYERS["silver"]["clean"]["domains"]["toji_clean"]["paths"]["parquet"]
))

partition_cols = LAYERS["silver"]["clean"]["domains"]["toji_clean"].get(
    "partition", ["region"]
)

print("[PATH] source =", toji_src_path)
print("[PATH] target =", toji_clean_path)
print("[CONF] partition =", partition_cols)

[PATH] source = /opt/spark/data/bronze/tojiSoyuJeongbo/parquet/year=2026/month=01
[PATH] target = /opt/spark/data/silver/clean/toji/year=2026/month=02
[CONF] partition = ['region']


# 브론즈(Raw) 데이터 로드

In [12]:
# ============================================================
# LOAD
# ============================================================
toji_df = spark.read.parquet(toji_src_path)

print("=== raw schema ===")
toji_df.printSchema()

=== raw schema ===
root
 |-- 고유번호: long (nullable = true)
 |-- 법정동코드: long (nullable = true)
 |-- 법정동명: string (nullable = true)
 |-- 대장구분코드: long (nullable = true)
 |-- 대장구분명: string (nullable = true)
 |-- 지번: string (nullable = true)
 |-- 집합건물일련번호: long (nullable = true)
 |-- 건물동명: string (nullable = true)
 |-- 건물층명: string (nullable = true)
 |-- 건물호명: string (nullable = true)
 |-- 건물실명: string (nullable = true)
 |-- 공유인일련번호: long (nullable = true)
 |-- 기준연월: string (nullable = true)
 |-- 지목코드: long (nullable = true)
 |-- 지목: string (nullable = true)
 |-- 토지면적: double (nullable = true)
 |-- 공시지가: long (nullable = true)
 |-- 소유구분코드: string (nullable = true)
 |-- 소유구분: string (nullable = true)
 |-- 국가기관구분코드: string (nullable = true)
 |-- 국가기관구분: string (nullable = true)
 |-- 소유권변동원인코드: string (nullable = true)
 |-- 소유권변동원인: string (nullable = true)
 |-- 소유권변동일자: string (nullable = true)
 |-- 공유인수: long (nullable = true)
 |-- 데이터기준일자: string (nullable = true)
 |-- 원천시도시군구코드: double (nul

[Stage 20:================================================>       (13 + 2) / 15]

중복 고유번호 개수: 7307095


# 데이터 정제

In [5]:
# ============================================================
# CLEANING
# ============================================================


toji_clean_df = (
    toji_df

    .filter(
        F.col("소유구분코드") == "01" 
    )
        
    # -----------------------------
    # 1. 필요한 컬럼만 선택
    # -----------------------------
    .select(
        F.col("고유번호").cast("string"),
        F.col("법정동명"),
        F.col("지번"),
        F.col("지목"),
        F.col("토지면적").cast("double"),
        F.col("공유인수"),
        F.col("소유권변동일자"),
        F.col("region")
    )
        
    # -----------------------------
    # 2. 날짜 파싱
    # -----------------------------
    .withColumn(
        "소유권변동일자_dt",
        F.to_date("소유권변동일자", "yyyy-MM-dd")
    )
    
    # -----------------------------
    # 3. 고유번호별 최신 1건만 유지
    # -----------------------------
    .withColumn(
        "rn",
        F.row_number().over(
            Window.partitionBy("고유번호")
                  .orderBy(F.col("소유권변동일자_dt").desc_nulls_last())
        )
    )
    .filter(F.col("rn") == 1)

    # -----------------------------
    # 4. 지번 분리
    # -----------------------------
    .withColumn("본번", F.split("지번", "-").getItem(0))
    .withColumn(
        "부번",
        F.when(
            F.size(F.split("지번", "-")) > 1,
            F.split("지번", "-").getItem(1)
        )
    )
    .drop("rn", "소유권변동일자_dt", "지번")
)

print("=== clean schema ===")
toji_clean_df.printSchema()
print("row count:", toji_clean_df.count())

=== clean schema ===
root
 |-- 고유번호: string (nullable = true)
 |-- 법정동명: string (nullable = true)
 |-- 지목: string (nullable = true)
 |-- 토지면적: double (nullable = true)
 |-- 공유인수: long (nullable = true)
 |-- 소유권변동일자: string (nullable = true)
 |-- region: string (nullable = true)
 |-- 본번: string (nullable = true)
 |-- 부번: string (nullable = true)



[Stage 3:==========================================>                (5 + 2) / 7]

row count: 24943989


# S_clean 데이터 저장

In [6]:
# ============================================================
# SAVE (silver/clean/toji)
# ============================================================

(
    toji_clean_df
    .write
    .mode("overwrite")
    .partitionBy(*partition_cols)
    .parquet(toji_clean_path)
)

print("✅ saved:", toji_clean_path)

✅ saved: /opt/spark/data/silver/clean/toji/year=2026/month=02


In [13]:
spark.stop()